1. Nuget Install

In [ ]:
#r "nuget:Microsoft.ML,*-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget:Microsoft.ML.FastTree, *-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22504.6 Microsoft.ML, 2.0.0-preview.22313.1 Microsoft.ML.FastTree, 2.0.0-preview.22313.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Microsoft.DotNet.Interactive.ExtensionLab.dll`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

ExplainCode 
 This extension generates Sequence diagrams from csharp code using Mermaid kernel.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;

using System.IO;
using System.Text;

2. Data Models

In [ ]:
public class HousingData
{
    [LoadColumn(0)]
    public float CrimeRate {get;set;}

    [LoadColumn(1)]
    public float Zoned {get;set;}

    [LoadColumn(2)]
    public float Proportion {get;set;}

    [LoadColumn(3)]
    public float RiverCoast {get;set;}

    [LoadColumn(4)]
    public float NOConcentration {get;set;}

    [LoadColumn(5)]
    public float NumOfRoomsPerDwelling {get;set;}

    [LoadColumn(6)]
    public float Age {get;set;}

    [LoadColumn(7)]
    public float EmployCenterDistance {get;set;}

    [LoadColumn(8)]
    public float HighwayAccessabilityRadius {get;set;}

    [LoadColumn(9)]
    public float TaxRate {get;set;}

    [LoadColumn(10)]
    public float PTRatio {get;set;}

    [LoadColumn(11)]
    public float MedianPrice {get;set;}
}

public class PricePredictions
{
    //Probability/estimations of continous values are found in Score column - ML.Net
    [ColumnName("Score")]
    public float MedianPrice {get;set;}
}

3. Load and Split Data

In [ ]:
var mlContext = new MLContext();
var trainingDataView = mlContext.Data.LoadFromTextFile<HousingData>("../../ML.Net.Training/Datasets/boston_housing.csv", hasHeader: true, separatorChar: ',');
var dataSplit = mlContext.Data.TrainTestSplit(trainingDataView, testFraction: 0.2);

In [ ]:
dataSplit.TestSet.ToTabularDataResource().Display();

4. Train Model

In [ ]:
var model = mlContext.Regression.Trainers.FastForest(
    numberOfLeaves: 50,
    numberOfTrees: 300
);

var pipeline = mlContext.Transforms.CopyColumns("Label", nameof(HousingData.MedianPrice))
                .Append(mlContext.Transforms.Categorical.OneHotEncoding("RiverCoast"))
                .Append(mlContext.Transforms.Concatenate("Features", 
                    nameof(HousingData.CrimeRate),
                    nameof(HousingData.Zoned),
                    nameof(HousingData.Proportion),
                    nameof(HousingData.RiverCoast),
                    nameof(HousingData.NOConcentration),
                    nameof(HousingData.NumOfRoomsPerDwelling),
                    nameof(HousingData.Age),
                    nameof(HousingData.EmployCenterDistance),
                    nameof(HousingData.HighwayAccessabilityRadius),
                    nameof(HousingData.TaxRate),
                    nameof(HousingData.PTRatio),
                    nameof(HousingData.MedianPrice)))
                .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
                .Append(model);

var trainedModel = pipeline.Fit(dataSplit.TrainSet);

5. Evaluate Model

In [ ]:
var testSet = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.Regression.Evaluate(testSet);

In [ ]:
metrics

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
16.915829084192712,1006.0162184502979,31.71775872362828,1006.0161997771356,0.9123524910597205


6. Save Model

In [ ]:
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./RandomForestRegressionModel.mdl");

7. Load and Predict Model

In [ ]:
var newSample = new HousingData
{
    Age = 58.7f,
    CrimeRate = 0.02985f,
    EmployCenterDistance = 6.0622f,
    HighwayAccessabilityRadius = 3f,
    NOConcentration = 0.458f,
    NumOfRoomsPerDwelling = 6.43f,
    Proportion = 2.18f,
    PTRatio = 18.7f,
    RiverCoast = 0,
    TaxRate = 222f,
    Zoned = 0f
};

using (var stream = new FileStream("./RandomForestRegressionModel.mdl", FileMode.Open, FileAccess.Read, FileShare.Read))
{
    var loadedModel = mlContext.Model.Load(stream, out _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<HousingData, PricePredictions>(loadedModel);
    
    var results = predictionEngine.Predict(newSample);

    Console.WriteLine(results.MedianPrice);
}

197.47504
